In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense,Input,Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd


In [3]:
df = pd.read_csv("D:/public_dat/train.csv")
df['Sentence'] = df['Sentence'].str.lower()

In [4]:
df.head()

,Id,Sentence,Entity_1,Entity_2,Relation
0,1,rising interest rates affects borrowing costs ...,Interest Rates,borrowers,affects
1,3,market volatility affects traders by leading t...,Market Volatility,Traders,affects
2,4,strong corporate earnings affect shareholders ...,Corporate Earnings,Shareholders,affects
3,5,regulatory changes affect financial institutio...,Regulatory Changes,Financial Institutions,affects
4,6,investor behavior affects market trends and as...,Investor Behavior,Market Trends,affects


In [5]:
df['Sentence']

0       rising interest rates affects borrowing costs ...
1       market volatility affects traders by leading t...
2       strong corporate earnings affect shareholders ...
3       regulatory changes affect financial institutio...
4       investor behavior affects market trends and as...
                              ...                        
7687    sharp, which sells aquos smartphones, fell zer...
7688    short gross sales come when an investor sells ...
7689    homeowners have a good chance of keeping their...
7690    german broker lang & schwarz tradecenter ag, w...
7691    global funds sold more south korean stocks tha...
Name: Sentence, Length: 7692, dtype: object

In [6]:

sentences=[]
for jok in df['Sentence']:
    sentences.append(jok)
word1=[]
for jok in df['Entity_1']:
    word1.append(jok)
word2=[]
for jok in df['Entity_2']:
    word2.append(jok)
relations=[]
for jok in df['Relation']:
    relations.append(jok)
    
# Separate data into input (sentences) and output (relations)
#sentences = np.array(sentences)
#word1 = np.array([df['Entity_1']])
#word2 = np.array([df['Entity_2']])
#relations = np.array([df['Relation']])
#sentences=sentences.flatten()
#word1=word1.flatten()
#word2=word2.flatten()
#relations=relations.flatten()
#print(relations)
# Tokenize sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences + word1 + word2)
vocab_size = len(tokenizer.word_index) + 1

# Encode sentences and relations
X_sentences = tokenizer.texts_to_sequences(sentences)
X_word1 = tokenizer.texts_to_sequences(word1)
X_word2 = tokenizer.texts_to_sequences(word2)
max_len = 128 #max(len(seq) for seq in X_sentences)
X_sentences = pad_sequences(X_sentences, maxlen=max_len)
X_word1 = pad_sequences(X_word1, maxlen=1)
X_word2 = pad_sequences(X_word2, maxlen=1)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(relations)  # Assuming relations are categorical strings


In [7]:

# Split data into train and test sets
X_sentences_train, X_sentences_test, X_word1_train, X_word1_test, X_word2_train, X_word2_test, y_train, y_test = train_test_split(X_sentences, X_word1, X_word2, y, test_size=0.1, random_state=6)

# Define LSTM model with named input layers
embedding_dim = 50
num_classes = len(set(relations))

input_sentences = Input(shape=(max_len,), name='input_sentences')
#input_word1 = Input(shape=(1,), name='input_word1')
#input_word2 = Input(shape=(1,), name='input_word2')

embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)
lstm_layer = LSTM(units=256, dropout=0.0, recurrent_dropout=0.0)
dense_layer = Dense(num_classes, activation='softmax')

embedding_sentences = embedding_layer(input_sentences)
#embedding_word1 = embedding_layer(input_word1)
#embedding_word2 = embedding_layer(input_word2)

lstm_output_sentences = lstm_layer(embedding_sentences)
#lstm_output_word1 = lstm_layer(embedding_word1)
#lstm_output_word2 = lstm_layer(embedding_word2)

concatenated = Concatenate()([lstm_output_sentences])
output = dense_layer(concatenated)

In [8]:

print("Shape of X_sentences_train:", X_sentences_train.shape)
print("Shape of X_sentences_test:", X_sentences_test.shape)
print(X_sentences_train)
max_len

Shape of X_sentences_train: (6922, 128)
Shape of X_sentences_test: (770, 128)
[[    0     0     0 ...   274 16546     7]
 [    0     0     0 ...     7    75   247]
 [    0     0     0 ...   200    41 17009]
 ...
 [    0     0     0 ...  1963  7146     7]
 [    0     0     0 ...   343    13  2850]
 [    0     0     0 ...    85     4   484]]


128

In [10]:

model = Model(inputs=[input_sentences], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_sentences_train, y_train, epochs=20, batch_size=30, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate({"input_sentences": X_sentences_test}, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/20
185/185 [==============================] - 7s 18ms/step - loss: 2.3433 - accuracy: 0.1611 - val_loss: 2.2440 - val_accuracy: 0.2123
Epoch 2/20
185/185 [==============================] - 3s 15ms/step - loss: 1.7464 - accuracy: 0.3867 - val_loss: 1.7673 - val_accuracy: 0.4101
Epoch 3/20
185/185 [==============================] - 3s 19ms/step - loss: 1.0139 - accuracy: 0.6942 - val_loss: 1.0150 - val_accuracy: 0.6982
Epoch 4/20
185/185 [==============================] - 3s 16ms/step - loss: 0.4763 - accuracy: 0.8701 - val_loss: 0.8466 - val_accuracy: 0.7877
Epoch 5/20
185/185 [==============================] - 3s 17ms/step - loss: 0.3891 - accuracy: 0.8992 - val_loss: 0.8897 - val_accuracy: 0.7812
Epoch 6/20
185/185 [==============================] - 3s 16ms/step - loss: 0.2237 - accuracy: 0.9473 - val_loss: 0.7933 - val_accuracy: 0.8108
Epoch 7/20
185/185 [==============================] - 3s 16ms/step - loss: 0.1207 - accuracy: 0.9686 - val_loss: 0.7757 - val_accuracy: 0.8173

In [11]:
df2 = pd.read_csv("D:/public_dat/test.csv")
df2['Sentence'] = df2['Sentence'].str.lower()

In [12]:
sentences2=[]
for jok in df2['Sentence']:
    sentences2.append(jok)
word12=[]
for jok in df2['Entity_1']:
    word12.append(jok)
word22=[]
for jok in df2['Entity_2']:
    word22.append(jok)
    
# Separate data into input (sentences) and output (relations)
#sentences = np.array(sentences)
#word1 = np.array([df['Entity_1']])
#word2 = np.array([df['Entity_2']])
#relations = np.array([df['Relation']])
#sentences=sentences.flatten()
#word1=word1.flatten()
#word2=word2.flatten()
#relations=relations.flatten()
#print(relations)
# Tokenize sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences + word1 + word2)
vocab_size = len(tokenizer.word_index) + 1

# Encode sentences and relations
X_sentences2 = tokenizer.texts_to_sequences(sentences2)
X_word12 = tokenizer.texts_to_sequences(word12)
X_word22 = tokenizer.texts_to_sequences(word22)
max_len = 128
X_sentences2 = pad_sequences(X_sentences2, maxlen=max_len)
X_word12 = pad_sequences(X_word12, maxlen=1)
X_word22 = pad_sequences(X_word22, maxlen=1)

In [13]:

X_sentences_text2 = tokenizer.sequences_to_texts(X_sentences2)
print(X_sentences_text2)

["the company also couldn ' t demonstrate that it receive perpetually verified that suppliers had made and tested repairs to equipment that crataegus oxycantha dissemble nuclear safety the theme say", 'tax evasion affects government revenue collection', 'deutsche bank the biggest currency trader said oct 29 that a flat market was negatively affecting fx revenues while hsbc said this week that revenue from foreign exchange tumbled 10 percent in the third quarter from a year earlier', 'welfare food stamps and programs for low income children could be affected by a lengthy shutdown', "property tax wage hike involve ' annual expenses", 'brand positioning baby food sales in september were 40 percent of the july level in the markets affected by the product recalls terisse said on the call', "federal contractors workers at america's top federal contractors were among those affected by the lapse in appropriations", "a surge in credit card usage affects credit card companies' profits", "you eit

In [14]:
df2['Id']

0       8920
1         64
2       2755
3       6871
4       7305
        ... 
1315    8344
1316    5943
1317    4424
1318    4457
1319    6012
Name: Id, Length: 1320, dtype: int64

In [15]:
pred = model.predict({"input_sentences": X_sentences2})
predicted_labels = np.argmax(pred, axis=1)
predicted_labels_text = label_encoder.inverse_transform(predicted_labels)

# Display test input sentences and corresponding predicted labels

for sentence,senteces1, predicted_label in zip(word12,word22, predicted_labels_text):
  
    print(f"Input word1: {sentence}")
    print(f"Input word2: {senteces1}")
    print(f"Predicted Label: {predicted_label}")
    print()
data={'Id':df2['Id'],'Relation':predicted_labels_text}
dfk=pd.DataFrame(data)

42/42 [==============================] - 1s 11ms/step
Input word1: Equipment
Input word2: Safety
Predicted Label: provides

Input word1: tax evasion
Input word2: government revenues
Predicted Label: affects

Input word1: flat market
Input word2: FX revenues
Predicted Label: affects

Input word1: lengthy shutdown
Input word2: welfare, food stamps and programs
Predicted Label: affects

Input word1: Property tax
Input word2: Annual expenses
Predicted Label: provides

Input word1: product recalls
Input word2: sales
Predicted Label: affects

Input word1: lapse in appropriations
Input word2: Federal Contractors
Predicted Label: affects

Input word1: Credit card
Input word2: Credit card companies' profits
Predicted Label: affects

Input word1: industrial users
Input word2: Investment Program
Predicted Label: affects

Input word1: shutdown
Input word2: civilian workers at the service academies
Predicted Label: affects

Input word1: cancellation notices
Input word2: population
Predicted Label: 

In [16]:
display(dfk)

,Id,Relation
0,8920,provides
1,64,affects
2,2755,affects
3,6871,affects
4,7305,provides
...,...,...
1315,8344,sells
1316,5943,offers
1317,4424,sells
1318,4457,sells


In [20]:

# saving the dataframe
dfk.to_csv(r'D:\Lithin\codlang\Submission.csv')